In [10]:
# Importar librerias
import requests
import json
import pandas as pd
import pymysql

In [11]:
# Api URL
Monedas = requests.get("http://economia.awesomeapi.com.br/json/last/USD-BRL,EUR-BRL,BTC-BRL")
Monedas = Monedas.json()

# Print json
# Monedas

In [12]:
# Convertir diccionario en una tabla
df = pd.DataFrame.from_dict(Monedas, orient = 'index').reset_index()
df

,index,code,codein,name,high,low,varBid,pctChange,bid,ask,timestamp,create_date
0,USDBRL,USD,BRL,Dólar Americano/Real Brasileiro,5.8079,5.79602,-0.001,-0.017221,5.8047,5.8077,1745296316,2025-04-22 01:31:56
1,EURBRL,EUR,BRL,Euro/Real Brasileiro,6.69344,6.65779,0.008933,0.133729,6.68896,6.70691,1745296308,2025-04-22 01:31:48
2,BTCBRL,BTC,BRL,Bitcoin/Real Brasileiro,517726,504465,4927,0.968,513705,513706,1745296404,2025-04-22 01:33:24


In [13]:
# Especificar Columnas
df_moneda = df.loc[:, ['code','codein','bid','ask','create_date']]

# Print table
df_moneda

,code,codein,bid,ask,create_date
0,USD,BRL,5.8047,5.8077,2025-04-22 01:31:56
1,EUR,BRL,6.68896,6.70691,2025-04-22 01:31:48
2,BTC,BRL,513705,513706,2025-04-22 01:33:24


In [14]:
# Renombrar Columnas
df_moneda.rename(columns = {'code':'moneda_base', 'codein':'moneda_destino', 'bid':'valor_compra', 'ask':'valor_venda', 'create_date':'data_hora'}, inplace = True)

# Print table
df_moneda

,moneda_base,moneda_destino,valor_compra,valor_venda,data_hora
0,USD,BRL,5.8047,5.8077,2025-04-22 01:31:56
1,EUR,BRL,6.68896,6.70691,2025-04-22 01:31:48
2,BTC,BRL,513705,513706,2025-04-22 01:33:24


In [15]:
# Conectar a base de datos mysql en docker desktop
connection = pymysql.connect(host = 'localhost',
                             user = 'myuser',
                             password = 'mypassword',
                             db = 'mydatabase')

# Crear cursor
cursor=connection.cursor()

In [16]:
# Crear lista de columnas para insertar
cols = ",".join([str(i) for i in df_moneda.columns.tolist()])

# Insertar registros DataFrame uno por uno
for i,row in df_moneda.iterrows():
    sql = "INSERT INTO Extracion_Monedas (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

# Confirmar conexion para guardar cambios
connection.commit()

In [17]:
# Ejecutar query
sql = "SELECT * FROM `Extracion_Monedas`"
cursor.execute(sql)

# Obtener todos los registros
result = cursor.fetchall()
for i in result:
    print(i)

('USD', 'BRL', 5.8049, 5.8079, datetime.datetime(2025, 4, 22, 1, 15, 43))
('EUR', 'BRL', 6.68896, 6.70691, datetime.datetime(2025, 4, 22, 1, 15, 34))
('BTC', 'BRL', 513217.0, 513221.0, datetime.datetime(2025, 4, 22, 1, 15, 55))
('USD', 'BRL', 5.8053, 5.8083, datetime.datetime(2025, 4, 22, 1, 20, 33))
('EUR', 'BRL', 6.69344, 6.71141, datetime.datetime(2025, 4, 22, 1, 16, 19))
('BTC', 'BRL', 513816.0, 513817.0, datetime.datetime(2025, 4, 22, 1, 21, 3))
('USD', 'BRL', 5.8066, 5.8096, datetime.datetime(2025, 4, 22, 1, 22, 2))
('EUR', 'BRL', 6.69344, 6.71141, datetime.datetime(2025, 4, 22, 1, 21, 54))
('BTC', 'BRL', 513978.0, 513979.0, datetime.datetime(2025, 4, 22, 1, 23, 6))
('USD', 'BRL', 5.8062, 5.8092, datetime.datetime(2025, 4, 22, 1, 26, 6))
('EUR', 'BRL', 6.69344, 6.71141, datetime.datetime(2025, 4, 22, 1, 25, 58))
('BTC', 'BRL', 513596.0, 513597.0, datetime.datetime(2025, 4, 22, 1, 27, 9))
('USD', 'BRL', 5.8047, 5.8077, datetime.datetime(2025, 4, 22, 1, 31, 56))
('EUR', 'BRL', 6.68

In [18]:
# Al finalizar el proceso cerrar la conexion
connection.close()